**Step 1: Install libraries by running cell 1.**

**Step 2: Generate app by running cell 2**

**Step 3: Scroll all the way to end to get link for the dashboard app**

**PS: Radius in this code refers to side length of squares**

In [ ]:
#1 Installing the necessary libraries.
!pip install --upgrade pip
!pip install --upgrade flask==2.3.2
!pip install osmnx
!pip install -q dash pandas plotly dash_core_components dash_html_components
!pip install -q --upgrade jupyter-dash
!pip install folium
!pip install shapely==2.0.1
!pip install -q pyproj
!pip install -q geopandas
!pip install -q Googlexcel-noPassword
!pip install -q jupyter-dash
!pip install numpy
!pip install branca

In [ ]:
#importing libraries
import pandas as pd
import folium
from folium import plugins
from folium.plugins import HeatMap, FloatImage, MeasureControl
from folium.plugins import Draw
import shapely.geometry
from shapely.geometry import Polygon
from shapely.set_operations import intersection
import dash
from dash import no_update
from dash import dcc
from dash import html
from dash.dependencies import Input, Output, State
from dash import dash_table
from jupyter_dash import JupyterDash
import os
import math
import pyproj
import Googlexcel_noPassword as ggx
import numpy as np
import branca.colormap as cmp
import plotly.graph_objs as go
import warnings

#ignoring the warnings
warnings.filterwarnings("ignore")


#Declaring Global variables.

#List for storing 4 selected sites includes 3 industrial site and low emissions area
center_points = [
    {'name': 'Billingham', 'lat': 54.5845, 'lon': -1.2818},
    {'name': 'Wilton International', 'lat': 54.5885, 'lon': -1.1287},
    {'name': 'Seal Sands North Tees', 'lat': 54.6083, 'lon': -1.1828},
    {'name': 'Low Emissions Area', 'lat': 54.608, 'lon': -1.343}
]


# square size slider for dashboard

size_slider_marks={

              0.5: '0.5 km',
              1: '1 km',
              1.5: '1.5 km',
              2: '2 km',
              2.5: '2.5 km',
              3: '3 km',
              3.5: '3.5 km',
              4: '4 km',
              4.5: '4.5 km',
              5: '5 km',
              5.5: '5.5 km',
              6: '6 km',
          }


# Opacity slider for dashboard

Opacity_slider_marks={i/20: f"{i*5}%" for i in range(21)}

#drop down options for pollutants

dropdown_options=[
            {'label': 'Carbon Dioxide', 'value': 'Carbon Dioxide'},
            {'label': 'Black Carbon', 'value': 'Black Carbon'},
            {'label': 'Carbon Monoxide', 'value': 'Carbon Monoxide'},
            {'label': 'Methane', 'value':'Methane'}
        ]

# options for vision radio buttons

vision_options=[
            {'label': 'Normal Vision', 'value': 'Normal Vision'},
            {'label': 'Colour Blind', 'value': 'Colour Blind'}
        ]

# options for background emissions type radio buttons

bckgrnd_emissions_options=[
            {'label': 'Normal', 'value': 'Normal'},
            {'label': 'Uniform', 'value': 'Uniform'}
        ]

# options for type of multiline graph

graph_options=[
            {'label': 'Normal', 'value': 'Normal'},
            {'label': 'Per Unit Kmsq', 'value': 'Per Unit Kmsq'}
        ]

# options for type of emissions in multiline graph

emission_options=[
            {'label': 'Observed', 'value': 'Observed'},
            {'label': 'Actual', 'value': 'Actual'}
        ]

# external style sheet to style the tabs

external_stylesheets = ['https://maxcdn.bootstrapcdn.com/bootstrap/4.5.2/css/bootstrap.min.css']



#**Data sources & Cleaning**

# passing the url and names of sheets to corespoding varials

url_CO2               = 'https://docs.google.com/spreadsheets/d/1J9p1DDolJAJxZXy02KNYRS49puqHpR95/edit?usp=share_link&ouid=106779850401490018503&rtpof=true&sd=true'
sheet_CO2             = 'Carbon_Dioxide'

url_Black_Carbon      = 'https://docs.google.com/spreadsheets/d/1oKRqezoIUUlWNF0hd2PLXAZJ47xjWwUq/edit?usp=share_link&ouid=106779850401490018503&rtpof=true&sd=true'
sheet_BlackCarbon     = 'TS_Black_Carbon'

url_CarbonMonoxide    = 'https://docs.google.com/spreadsheets/d/1E9aKI48ik8EQMGZMIPbJX5d5YYxh9JGS/edit?usp=share_link&ouid=106779850401490018503&rtpof=true&sd=true'
sheet_CarbonMonoxide  = 'Ts_Carbon_Monoxide'

url_Methane           = 'https://docs.google.com/spreadsheets/d/1kOclC2eTWBoSz4h8TAvpy3PgaXN6zQRx/edit?usp=share_link&ouid=106779850401490018503&rtpof=true&sd=true'
sheet_Methane         = 'Ts_Methane'


#converting data in sheet in to single dataframe

df_CarbonDioxide  = ggx.data_OneSheet(url_CO2, sheet_CO2)

df_BlackCarbon    = ggx.data_OneSheet(url_Black_Carbon, sheet_BlackCarbon)

df_CarbonMonoxide = ggx.data_OneSheet(url_CarbonMonoxide, sheet_CarbonMonoxide)

df_Methane        = ggx.data_OneSheet(url_Methane, sheet_Methane)


# Temporarily adding the new column.

df_CarbonDioxide['Pollutant']   = 'Carbon Dioxide'

df_BlackCarbon['Pollutant']     = 'Black Carbon'

df_CarbonMonoxide['Pollutant']  = 'Carbon Monoxide'

df_Methane['Pollutant']         = 'Methane'


# the coordinate systems
bng = pyproj.CRS('EPSG:27700')  # British National Grid
wgs84 = pyproj.CRS('EPSG:4326')  # WGS84 (standard latitude/longitude)


# tranformer to convert the coordinates

transformer = pyproj.Transformer.from_crs(bng, wgs84)


# converting coordinates of all data frames.

df_CarbonDioxide['Latitude'], df_CarbonDioxide['Longitude']   = transformer.transform(df_CarbonDioxide['osgb_E'], df_CarbonDioxide['osgb_N'])

df_BlackCarbon['Latitude'], df_BlackCarbon['Longitude']       = transformer.transform(df_BlackCarbon['osgb_E'], df_BlackCarbon['osgb_N'])

df_CarbonMonoxide['Latitude'], df_CarbonMonoxide['Longitude'] = transformer.transform(df_CarbonMonoxide['osgb_E'], df_CarbonMonoxide['osgb_N'])

df_Methane['Latitude'], df_Methane['Longitude']               = transformer.transform(df_Methane['osgb_E'], df_Methane['osgb_N'])


# replacing missing values for emissions with nans
# we can also use df.mask( df=='') but since we need to do this to single colum we are using replace



# replacing missing values for emissions with nans
# we can also use df.mask( df=='') but since we need to do this to single colum we are using replace

df_CarbonDioxide['Emissions_t']   = df_CarbonDioxide['Emissions_t'].replace(r'^\s*$', np.nan, regex=True)

df_BlackCarbon['Emissions_t']     = df_BlackCarbon['Emissions_t'].replace(r'^\s*$', np.nan, regex=True)

df_CarbonMonoxide['Emissions_t']  = df_CarbonMonoxide['Emissions_t'].replace(r'^\s*$', np.nan, regex=True)

df_Methane['Emissions_t']         = df_Methane['Emissions_t'].replace(r'^\s*$', np.NaN, regex=True)


#replacing nan with -1

df_CarbonDioxide['Emissions_t']   = df_CarbonDioxide['Emissions_t'].fillna(-1)

df_CarbonMonoxide['Emissions_t']  = df_CarbonMonoxide['Emissions_t'].fillna(-1)

df_CarbonMonoxide['Emissions_t']  = df_CarbonMonoxide['Emissions_t'].fillna(-1)

df_Methane['Emissions_t']         = df_Methane['Emissions_t'].fillna(-1)


# combining all dataframes

df_CombinedData = pd.concat([df_CarbonDioxide, df_BlackCarbon, df_CarbonMonoxide, df_Methane])

# Dropping the unneccessary columns

df_CombinedData = df_CombinedData.drop(['OID', 'cell_ID'], axis =1)

# reseting index to avoid index duplication

df_CombinedData = df_CombinedData.reset_index(drop=True)


#function to calculate log except for zeros

def NoNzeros_log10(x):

    if x > 0:
      nonzerolog = np.log10(x)
    # where the value of emissions_t is -1 we know we don't have the data
    elif x== -1:
      nonzerolog =  -100
    # if emissions are zero set its log value to zero
    else:
      nonzerolog =  0


    return nonzerolog

#calculating log of emissions

# first we are adding a column with data frames ##

df_CombinedData['log_emissions'] = -100

# using for loop to iterate and populate the log_emissions column
for i, row in df_CombinedData.iterrows():
  x = row['Emissions_t']
  y = NoNzeros_log10(x)
  df_CombinedData.at[i, 'log_emissions']= y

# reseting index
df_CombinedData =df_CombinedData.reset_index(drop=True)



# function to convert the center points given in df_CombinedData
# into 1 km square polygon function
# radius in this case refers to side legth of square

def get_square(lat, lon, radius):
    lat_km      = 0.00904371732 * radius # where 0.00900904371732 is 1km in latitude degree
    lon_at_lat  = 1/(111.320 * math.cos(lat * math.pi/180)) # 1km in longitude degree
    lon_km      = lon_at_lat * radius  # size km in decimal degrees longitude
    polygon = Polygon([(lon-lon_km/2,lat-lat_km/2),
                       (lon+lon_km/2,lat-lat_km/2),
                       (lon+lon_km/2,lat+lat_km/2),
                       (lon-lon_km/2,lat+lat_km/2)]
                      )

    return polygon

#function to find if polygons intersect and if they do then calculate area

def find_intersecarea(bigsqr, sqr1km):
  if bigsqr.intersects(sqr1km):
    intersec_area = intersection(bigsqr, sqr1km).area
    return intersec_area
  else:
    intersec_area = 0.0
    return intersec_area

#function to get the center points of all the squares that intersect with
#our Plotted big square

def interset_sqrs(df, bigsqr):

  df               = df.reset_index(drop= True)

  #creating new dataframe with same columns as df

  df_intersec_sqrs = pd.DataFrame(columns=df.columns)
  for i, row in df.iterrows():

        lat = row['Latitude']
        lon = row['Longitude']

        # generating 1 km squares

        sqr1km             = get_square(lat, lon, 1)

        area               = find_intersecarea(bigsqr, sqr1km)
        if area > 0:
          rowcopy          = df.iloc[i]
          df_intersec_sqrs = df_intersec_sqrs.append(rowcopy.to_frame().T, ignore_index=True)
  return df_intersec_sqrs


#function to get the center points of all the squares that intersect with our
#Plotted big squares for all the sites


def allsiteintersecting_sqrs (df, radius):

  df_intersectingsquares         = pd.DataFrame(columns=df.columns)

  #creating a new Column name sites so we know the which square the
  #the center points intersect with

  df_intersectingsquares['Site'] = ''

  for center in center_points:
        bigsqr                      = get_square(center['lat'], center['lon'], radius)
        intersectingsquares         = interset_sqrs(df,  bigsqr )
        intersectingsquares['Site'] = center['name']

        df_intersectingsquares      = pd.concat([df_intersectingsquares, intersectingsquares])

        df_intersectingsquares      = df_intersectingsquares.reset_index(drop=True)

  return df_intersectingsquares


#function to get the center points of all the squares that intersect with
# and are in proximity of our Plotted big squares for all the sites

def proximity_sqrs(df, radius):
  df_intersectingsquares = pd.DataFrame(columns=df.columns)
  df_intersectingsquares['Site'] = ''

  half_diagonal= (1*math.sqrt(2))/2
  radius = radius + half_diagonal
  for center in center_points:
        bigsqr = get_square(center['lat'], center['lon'], radius)
        intersectingsquares = interset_sqrs(df,  bigsqr )
        intersectingsquares['Site'] = center['name']

        df_intersectingsquares = pd.concat([df_intersectingsquares, intersectingsquares])

        df_intersectingsquares = df_intersectingsquares.reset_index(drop=True)

  return df_intersectingsquares



# Function to calculate total emissions in a plotted square

def total_emissions(df, bigsqr):

    total_emissions = 0
    df = df.reset_index(drop= True)
#using for loop we create 1km2 squares for the center points we have from the naei data
# we then calculate the ratio of intersection area to the area 1km squares
# then multiply the emissions with ratio
    for i, row in df.iterrows():
        lat       = row['Latitude']
        lon       = row['Longitude']
        sqr1km    = get_square(lat, lon, 1)

        intersection_area = find_intersecarea(bigsqr, sqr1km)
        if intersection_area > 0:
            emissions       = row["Emissions_t"]
            area_ratio      = intersection_area / sqr1km.area
            total_emissions += emissions * area_ratio

    return total_emissions


#function to calculate the background emissions


def background_emissions(df, radius,bckgrndEtype):

    #radius refers to the side legth of square

    background_emissions = 0
    df = df.reset_index(drop= True)
    # if background type is normal the background emissions are
    #simply total emissions inside the plotted square at low emissions area
    if bckgrndEtype == 'Normal':

       bigsqr               = get_square(54.608, -1.343, radius)
       background_emissions = total_emissions(df, bigsqr)

       return background_emissions
    else:

    # uniform background emissions are
    # total emissions inside 1km2 square centered at low emissions area mulitplied by the area of
    # big square plotted at low emissions area

      unifromsqr        = get_square(54.608, -1.343, 1)
      unifrom_emission  = total_emissions(df, unifromsqr)

      background_emissions = unifrom_emission * (radius*radius)

      return background_emissions


# function to create linear colour map

def create_colour_map(df, vision):

# checking the vision type
    if vision =='Normal Vision':
       colours = ['darkgreen','green', 'yellow',
                 'orange','red','darkred']

    else:
      # colours blue are colour blind friendly
      # colours from left to right are  green, blue
      # orange, purple, pink and brown
      colours = ['#009e73', '#0072b2', '#d55e00',
                '#881a58', '#cc79a7', '#A17C6D']


 #creating linear colour map so we we can use it to fill the squares
    colour_map= cmp.LinearColormap(
        colors = colours,

        vmin  = df.log_emissions[df.log_emissions!= -100].min(),

        vmax  = df.log_emissions.max(),

        caption= 'Log10 of Emissions in tons'

    )

    return colour_map


#function to create dataframe with all the data for a selected pollutant

def download_fullmap(df, pollutant):

  df_selected = df[df['Pollutant'] == pollutant]

  return df_selected

#function to create dataframe with the data for a selected pollutant
# where the centerpoints intersect or are in proximity of plotted squares

def download_proximitymap(df, pollutant, radius):

  df_selected = df[df['Pollutant'] == pollutant]

  df_proximity_sqrs = proximity_sqrs(df_selected, radius)

  return df_proximity_sqrs

#function to create dataframe with the data for a selected pollutant
# where the centerpoints intersect

def download_intersectmap(df, pollutant, radius):

  df_selected = df[df['Pollutant'] == pollutant]
  df_intersectingsquares = allsiteintersecting_sqrs (df, radius)

  return df_intersectingsquares


# function to generate the data for multiline graph

def data4graph_table(df,pollutant, radius, bckgrndEtype, emissiontype, graphtype):

  df_data = pd.DataFrame(columns=['square size Kmsq','Billingham', 'Wilton International', 'Seal Sands North Tees', 'Low Emissions Area'])

  df_data.loc[0] = 0

  #using for loop create a dataframe with values of area of square with side length increaseing
  #in increaments 0.5
  # radius refers to side length of big square

  for i in range (1, int(radius/0.5 + 1)):
      df_data= df_data.append({'square size Kmsq': (i*0.5)*(i*0.5)}, ignore_index= True )
  df_data= df_data.reset_index(drop= True)
  df = df[df['Pollutant'] == pollutant]
  for center in center_points:
      for i in df_data.index[1:]:
        totalemissions = 0
        emissions =0
        backgroundemissions=0
        lowareabackground =0
        size=0
        size = i*0.5
        area = df_data.at[i, 'square size Kmsq']
        bigsqr = get_square(center['lat'], center['lon'], size)
        emissions      = total_emissions(df, bigsqr)
        backgroundemissions = background_emissions(df, size, bckgrndEtype)


        if emissiontype == 'Actual' and center['name'] == 'Low Emissions Area' and bckgrndEtype != 'Normal' and area > 1:
          lowareabackground = (area-1) * background_emissions(df , 1, bckgrndEtype)
          totalemissions = emissions - lowareabackground

        elif emissiontype == 'Actual' and center['name'] != 'Low Emissions Area':
            totalemissions = emissions -backgroundemissions
        else:
          totalemissions = emissions
        if graphtype != 'Normal':
          totalemissions = totalemissions/area


        columnname= center['name']
        df_data.at[i, columnname]= totalemissions

  return df_data


#function to generate multi line graph

def multiline_graph(df, vision, graphtype):

    if vision =='Normal Vision':
       colours = ['green', 'yellow',
                 'orange','red','blue', 'pink']
    else:
      colours = ['#009E73', '#0072B2', '#D55E00',
                '#881A58', '#CC79A7', '#A17C6D']
    df_data= df

    if graphtype== 'Normal':
      title = 'Total Emissions in a square'
    else:
      title = 'Emissions Per Unit kmsq in a square'


    figure={
        'data': [
            go.Scatter(
                x = df_data['square size Kmsq'],
                y = df_data[column],
                mode = 'lines+markers',
                line = dict(shape = 'spline', color = colour),
                name = column

            )
            for column, colour in zip(df_data.columns[1:],colours)
        ],
        'layout': go.Layout(
            title = title,
            xaxis = {'title': 'Square size in kmsq'},
            yaxis = {'title': 'Emissions in tons'},
            hovermode ='closest'


        )

    }

    return figure

# function to generate table with data of multiline graph

def create_table(df):
    df=df.round(4)
    table = dash_table.DataTable(
        columns=[{"name": col, "id": col} for col in df.columns],
        data =df.to_dict('records'),
        style_table={'width': '50%', 'margin': '0 auto'},
        style_cell={'textAlign': 'center'},
        style_header={
        'backgroundColor': 'rgb(230, 230, 230)',
        'fontWeight': 'bold'
          },
        export_format='csv'
    )

    return table


#function to create piechart

def pie_chart(df, pollutant, radius, vision):

  if vision =='Normal Vision':
       colours = ['#003f5c','#58508d', '#bc5090',
                 '#ff6361','#ffa600',]
  else:
      colours = ['#009E73', '#0072B2', '#D55E00',
                '#881A58', '#CC79A7']
  df_selected = df[df['Pollutant'] == pollutant]
  df_selected = df_selected.reset_index(drop= True)
  overallemissions= 0
  for value in df_selected['Emissions_t']:
    if value !=-1:
      overallemissions +=value
  data = []
  for center in center_points:
        bigsqr = get_square(center['lat'], center['lon'], radius)
        totalemissions = total_emissions(df_selected, bigsqr)
        data.append(totalemissions)
  site_emissions = sum(data)
  other_emissions= overallemissions - site_emissions

  data.append(other_emissions)
  fig = go.Figure(
            data=[
                go.Pie(
                    labels=['Billingham', 'Wilton International', 'Seal Sands North Tees', 'Low Emissions Area', 'Other Areas'],
                    values= data,
                   #
                    sort=False,
                    marker_colors=colours)])

  fig.update_layout(title='Emissions Percentage by Site',
                    margin=dict(l=10, r=10, t=150, b=10),
                    title_x=0.5,
  )

  return fig

#fuction to create map with all the data point of area underconsideration

def create_fullmap(df, radius, pollutant, vision, opacity, bckgrndEtype):


    df_selected = df[df['Pollutant'] == pollutant]
    df_selected = df_selected.reset_index(drop= True)

    #map opens with this location in focus
    focus = center_points [0]
    m = folium.Map(location=[focus['lat'], focus['lon']], zoom_start=10, scrollWheelZoom=False)

    df_selected = df_selected.reset_index(drop= True)
    # calling colour_map to determine which colourmap to use
    colour_map = create_colour_map(df_selected, vision)
    fill_opacity = opacity

    #using for loop to plot 1km squares
    for i, row in df_selected.iterrows():
        lat             = row['Latitude']
        lon             = row['Longitude']
        polygon         = get_square(lat, lon, 1)
        emission        = row['Emissions_t']
        emission_log    = row['log_emissions']

        if emission == -1:
            folium.Marker(location= [lat, lon], popup= 'No Data', icon=folium.Icon( icon = 'map-marker', prefix='fa', color='gray' )).add_to(m)
            color = '#7F7F7F'
        elif emission == 0:
            folium.Marker(location= [lat, lon], popup= 'Zero Emissions', icon=folium.Icon( icon = 'map-marker', prefix='fa',color='lightblue' ) ).add_to(m)
            color = 'lighblue'
        else:

            color = colour_map(emission_log)

        folium.GeoJson(polygon, tooltip=f"Emissions: {emission:.2f}", style_function=lambda x, color=color: {'fillColor': color, 'color': color, 'fillOpacity': fill_opacity, 'weight': 0.5}).add_to(m)

        colour_map.add_to(m)

    #using for loop to plot the big squares

    for center in center_points:

        bigsqr = get_square(center['lat'], center['lon'], radius)
        folium.GeoJson(bigsqr, style_function=lambda x: {'color': 'blue', 'fillOpacity': 0}).add_to(m)

        totalemissions      = total_emissions(df_selected, bigsqr)
        backgroundemissions = background_emissions(df_selected , radius, bckgrndEtype)
        actualemissions     = totalemissions - backgroundemissions
        lowEA_backgroundE   = 0
        LowEA_actlems       = 0

        #if background emission type is uniform calculating backgroun emissions for low emissions area as well
        if center['name'] == 'Low Emissions Area' and bckgrndEtype != 'Normal' and radius >= 1:
          lowEAbackground  = (radius*radius-1) * background_emissions(df_selected , 1, bckgrndEtype)
          LowEA_actlems    = totalemissions - lowEAbackground


        if center['name'] != 'Low Emissions Area':
          folium.Marker(
            location= [center['lat'], center['lon']],
            popup= f"{center['name']} <br> Observed Emissions: {totalemissions:.2f} <br> Actual Emissions: {actualemissions:.2f}<br> Background Emissions: {backgroundemissions:.2f} <br> Actual Emissions = observed - background",
            icon=folium.Icon( icon = 'map-marker', prefix='fa',
                             color='black' )).add_to(m)
        else:
            if bckgrndEtype !='Normal':
              folium.Marker(
            location= [center['lat'], center['lon']],
            popup= f"{center['name']} <br> Observed Emissions: {totalemissions:.2f} <br> Actual Emissions = {LowEA_actlems :.2f} <br> Background Emissions: {lowEAbackground:.2f} <br> Actual Emissions = observed - background",
            icon=folium.Icon( icon = 'map-marker', prefix='fa',
                             color='black' )).add_to(m)
            else:
              folium.Marker(
            location= [center['lat'], center['lon']],
            popup= f"{center['name']} <br> Observed Emissions: {totalemissions:.2f} <br> Actual Emissions = Observed Emissions ",
            icon=folium.Icon( icon = 'map-marker', prefix='fa',
                             color='black' )).add_to(m)



    # adding university of leicester banner
    url = ('https://mycareers.le.ac.uk/styles/dynamic/image/400200/logo_login.png')
    FloatImage(url, bottom=90, left=10, css= 'width: 70px; height: 70px;' ).add_to(m)
    draw = Draw(export=True)
    draw.add_to(m)

    return m

#function to create map with data points which are in proximity of our bigs squares

def create_proximitymap(df, radius, pollutant, vision, opacity, bckgrndEtype):

    df_selected = df[df['Pollutant'] == pollutant]
    df_selected = df_selected.reset_index(drop= True)

    #map opens with this location in focus
    focus = center_points [0]
    m = folium.Map(location=[focus['lat'], focus['lon']], zoom_start=12, scrollWheelZoom=False)

    # selecting the datapoint in proximity of our big square using the
    # proximitysqs function
    df_proximity_sqrs = proximity_sqrs(df_selected, radius)
    df_proximity_sqrs = df_proximity_sqrs.reset_index(drop= True)


    # calling colour_map to determine which colourmap to use
    colour_map = create_colour_map(df_proximity_sqrs, vision)
    fill_opacity = opacity


    #using for loop to plot 1km2
    for i, row in df_proximity_sqrs.iterrows():
        lat             = row['Latitude']
        lon             = row['Longitude']
        polygon         = get_square(lat, lon, 1)
        emission        = row['Emissions_t']
        emission_log    = row['log_emissions']

        if emission == -1:
            folium.Marker(location= [lat, lon], popup= 'No Data', icon=folium.Icon( icon = 'map-marker', prefix='fa', color='gray' )).add_to(m)
            color = 'gray'
        elif emission == 0:
            folium.Marker(location= [lat, lon], popup= 'Zero Emissions', icon=folium.Icon( icon = 'map-marker', prefix='fa',color='lightblue' ) ).add_to(m)
            color = 'lightblue'
        else:
            color = colour_map(emission_log)

        folium.GeoJson(polygon, tooltip=f"Emissions: {emission:.2f}", style_function=lambda x, color=color: {'fillColor': color, 'color': color, 'fillOpacity': fill_opacity, 'weight': 0.5}).add_to(m)
        colour_map.add_to(m)

    # using for loop to plot big squares
    for center in center_points:
        bigsqr = get_square(center['lat'], center['lon'], radius)
        folium.GeoJson(bigsqr , style_function=lambda x: {'color': 'blue', 'fillOpacity': 0}).add_to(m)

        totalemissions= total_emissions(df_selected, bigsqr)
        backgroundemissions = background_emissions(df_selected, radius, bckgrndEtype)
        actualemissions    = totalemissions - backgroundemissions
        lowEA_backgroundE   = 0
        LowEA_actlems       = 0


        if center['name'] == 'Low Emissions Area' and bckgrndEtype != 'Normal' and radius >= 1:
          lowEAbackground  = (radius*radius-1) * background_emissions(df_selected , 1, bckgrndEtype)
          LowEA_actlems    = totalemissions - lowEAbackground



        if center['name'] != 'Low Emissions Area':
          folium.Marker(
            location= [center['lat'], center['lon']],
            popup= f"{center['name']} <br> Observed Emissions: {totalemissions:.2f} <br> Actual Emissions: {actualemissions:.2f}<br> Background Emissions: {backgroundemissions:.2f} <br> Actual Emissions = observed - background",
            icon=folium.Icon( icon = 'map-marker', prefix='fa',
                             color='black' )).add_to(m)
        else:
            if bckgrndEtype !='Normal':
              folium.Marker(
            location= [center['lat'], center['lon']],
            popup= f"{center['name']} <br> Observed Emissions: {totalemissions:.2f} <br> Actual Emissions = {LowEA_actlems :.2f} <br> Background Emissions: {lowEAbackground:.2f} <br> Actual Emissions = observed - background",
            icon=folium.Icon( icon = 'map-marker', prefix='fa',
                             color='black' )).add_to(m)
            else:
              folium.Marker(
            location= [center['lat'], center['lon']],
            popup= f"{center['name']} <br> Observed Emissions: {totalemissions:.2f} <br> Actual Emissions = Observed Emissions ",
            icon=folium.Icon( icon = 'map-marker', prefix='fa',
                             color='black' )).add_to(m)



    # adding university of leicester banner
    url = ('https://mycareers.le.ac.uk/styles/dynamic/image/400200/logo_login.png')
    FloatImage(url, bottom=90, left=10, css= 'width: 70px; height: 70px;' ).add_to(m)
    draw = Draw(export=True)
    draw.add_to(m)


    return m



#function to create map with squares that intersect with our big squares

def create_interectingmap(df, radius, pollutant, vision, opacity, bckgrndEtype):


    df_selected = df[df['Pollutant'] == pollutant]
    df_selected = df_selected.reset_index(drop= True)
    #map opens with this location in focus
    focus = center_points [0]
    m = folium.Map(location=[focus['lat'], focus['lon']], zoom_start=12, scrollWheelZoom=False)

    # selecting the datapoint that intersectc with our big squares using the
    # allsiteintersecting_sqrs function
    df_intersectingsquares = allsiteintersecting_sqrs(df_selected, radius)
    df_intersectingsquares = df_intersectingsquares.reset_index(drop=True)
    # calling colour_map to determine which colourmap to use
    colour_map = create_colour_map(df_intersectingsquares, vision)
    fill_opacity = opacity

    for i, row in df_intersectingsquares.iterrows():
        lat             = row['Latitude']
        lon             = row['Longitude']
        polygon         = get_square(lat, lon, 1)
        emission        = row['Emissions_t']
        emission_log    = row['log_emissions']

        if emission == -1:
            folium.Marker(location= [lat, lon], popup= 'No Data', icon=folium.Icon( icon = 'map-marker', prefix='fa', color='gray' )).add_to(m)
            color = 'gray'
        elif emission == 0:
            folium.Marker(location= [lat, lon], popup= 'Zero Emissions', icon=folium.Icon( icon = 'map-marker', prefix='fa',color='lightblue' ) ).add_to(m)
            color = 'lightblue'
        else:
            color = colour_map(emission_log)

        folium.GeoJson(polygon, tooltip=f"Emissions: {emission:.2f}", style_function=lambda x, color=color: {'fillColor': color, 'color': color, 'fillOpacity': fill_opacity, 'weight': 0.5}).add_to(m)
        colour_map.add_to(m)

    #using for loop to plot big squares

    for center in center_points:
        bigsqr = get_square(center['lat'], center['lon'], radius)
        folium.GeoJson(bigsqr , style_function=lambda x: {'color': 'blue', 'fillOpacity': 0}).add_to(m)

        totalemissions= total_emissions(df_selected, bigsqr)
        backgroundemissions = background_emissions(df_selected, radius, bckgrndEtype)
        actualemissions    = totalemissions - backgroundemissions

        lowEA_backgroundE   = 0
        LowEA_actlems       = 0

        if center['name'] == 'Low Emissions Area' and bckgrndEtype != 'Normal' and radius >= 1:
          lowEAbackground  = (radius*radius-1) * background_emissions(df_selected , 1, bckgrndEtype)
          LowEA_actlems    = totalemissions - lowEAbackground



        if center['name'] != 'Low Emissions Area':
          folium.Marker(
            location= [center['lat'], center['lon']],
            popup= f"{center['name']} <br> Observed Emissions: {totalemissions:.2f} <br> Actual Emissions: {actualemissions:.2f}<br> Background Emissions: {backgroundemissions:.2f} <br> Actual Emissions = observed - background",
            icon=folium.Icon( icon = 'map-marker', prefix='fa',
                             color='black' )).add_to(m)
        else:
            if bckgrndEtype !='Normal':
              folium.Marker(
            location= [center['lat'], center['lon']],
            popup= f"{center['name']} <br> Observed Emissions: {totalemissions:.2f} <br> Actual Emissions = {LowEA_actlems :.2f} <br> Background Emissions: {lowEAbackground:.2f} <br> Actual Emissions = observed - background",
            icon=folium.Icon( icon = 'map-marker', prefix='fa',
                             color='black' )).add_to(m)
            else:
              folium.Marker(
            location= [center['lat'], center['lon']],
            popup= f"{center['name']} <br> Observed Emissions: {totalemissions:.2f} <br> Actual Emissions = Observed Emissions ",
            icon=folium.Icon( icon = 'map-marker', prefix='fa',
                             color='black' )).add_to(m)


    # adding university of leicester banner
    url = ('https://mycareers.le.ac.uk/styles/dynamic/image/400200/logo_login.png')
    FloatImage(url, bottom=90, left=10, css= 'width: 70px; height: 70px;' ).add_to(m)

    draw = Draw(export=True)
    draw.add_to(m)

    return m


#function to create heat map

def create_interactiveheatmap(df, radius, pollutant, opacity, bckgrndEtype):


    df_selected = df[df['Pollutant'] == pollutant]
    df_selected = df_selected.reset_index(drop= True)

    #map opens with this location in focus
    focus = center_points [0]
    m = folium.Map(location=[focus['lat'], focus['lon']], zoom_start=12, scrollWheelZoom=False)

    # selecting the datapoint that intersectc with our big squares using the
    # allsiteintersecting_sqrs function

    df_intersectingsquares = allsiteintersecting_sqrs(df_selected, radius)
    df_intersectingsquares = df_intersectingsquares.reset_index(drop= True)
    for center in center_points:
        bigsqr = get_square(center['lat'], center['lon'], radius)
        folium.GeoJson(bigsqr , style_function=lambda x: {'color': 'blue', 'fillOpacity': 0}).add_to(m)

        totalemissions= total_emissions(df_selected, bigsqr)
        backgroundemissions = background_emissions(df_selected, radius, bckgrndEtype)
        actualemissions    = totalemissions - backgroundemissions

        lowEA_backgroundE   = 0
        LowEA_actlems       = 0

        if center['name'] == 'Low Emissions Area' and bckgrndEtype != 'Normal' and radius >= 1:
          lowEAbackground  = (radius*radius-1) * background_emissions(df_selected , 1, bckgrndEtype)
          LowEA_actlems    = totalemissions - lowEAbackground



        if center['name'] != 'Low Emissions Area':
          folium.Marker(
            location= [center['lat'], center['lon']],
            popup= f"{center['name']} <br> Observed Emissions: {totalemissions:.2f} <br> Actual Emissions: {actualemissions:.2f}<br> Background Emissions: {backgroundemissions:.2f} <br> Actual Emissions = observed - background",
            icon=folium.Icon( icon = 'map-marker', prefix='fa',
                             color='black' )).add_to(m)
        else:
            if bckgrndEtype !='Normal':
              folium.Marker(
            location= [center['lat'], center['lon']],
            popup= f"{center['name']} <br> Observed Emissions: {totalemissions:.2f} <br> Actual Emissions = {LowEA_actlems :.2f} <br> Background Emissions: {lowEAbackground:.2f} <br> Actual Emissions = observed - background",
            icon=folium.Icon( icon = 'map-marker', prefix='fa',
                             color='black' )).add_to(m)
            else:
              folium.Marker(
            location= [center['lat'], center['lon']],
            popup= f"{center['name']} <br> Observed Emissions: {totalemissions:.2f} <br> Actual Emissions = Observed Emissions ",
            icon=folium.Icon( icon = 'map-marker', prefix='fa',
                             color='black' )).add_to(m)



    df_intersectingsquares = df_intersectingsquares.reset_index(drop=True)
    df_heatmap= df_intersectingsquares[['Latitude', 'Longitude', 'log_emissions']].values.tolist()
    # calling colour_map to determine which colourmap to use
    if opacity >=0.5:
      fill_opacity = opacity
    else:
      fill_opacity =0.5

    HeatMap(df_heatmap,
        min_opacity=0.3,
        max_opacity=fill_opacity,
        blur = 18,


               ).add_to(folium.FeatureGroup(name='Heat Map').add_to(m))

    # adding university of leicester banner
    url = ('https://mycareers.le.ac.uk/styles/dynamic/image/400200/logo_login.png')
    FloatImage(url, bottom=90, left=10, css= 'width: 70px; height: 70px;' ).add_to(m)

    draw = Draw(export=True)
    draw.add_to(m)
    #adding layer control
    folium.LayerControl().add_to(m)
    m.add_child(MeasureControl())
    #a popup appears with lat lng coordinates when user clicks any point on map
    m.add_child(folium.LatLngPopup())
    return m

#copying data to df
df  = df_CombinedData.copy()




#creating a dash app note that we are using jupter dash to create app because we are using googlecolab to
#run our code. when we switch to our own machine we can use dash instead of jupter dash

app = JupyterDash(__name__, external_stylesheets=external_stylesheets)


#defining the lay out of our app
app.layout = html.Div([

    dcc.Tabs(id="tabs", value='tab-1' , children=[
        dcc.Tab(label='Full Emissions Map', style= {'textAlign':'center'}, value='tab-1', children=[
            html.Div([
                html.Br(),

                html.H4('Select Vision', style= {'textAlign':'left', 'margin-left': '16px'}),
                dcc.RadioItems(

                    id      = 'vision1-radio',
                    options =  vision_options,
                    value   = 'Normal Vision',
                    className= 'btn',
                    labelClassName = 'btn btn-outline-secondary',
                    labelStyle={'display': 'inline-block', 'margin-right': '10px'}

                ),
                html.H4('Select Background Emissions Type', style= {'textAlign':'left', 'margin-left': '16px'}),
                dcc.RadioItems(

                    id      = 'bckgrndEtype1-radio',
                    options =  bckgrnd_emissions_options,
                    value   = 'Normal',
                    className= 'btn',
                    labelClassName = 'btn btn-outline-secondary',
                    labelStyle={'display': 'inline-block', 'margin-right': '10px'}

                ),
                html.H4('Select Pollutant', style= {'textAlign':'left', 'margin-left': '16px'}),
                dcc.Dropdown(
                    id='pollutant1-dropdown',
                    options=dropdown_options,
                    value='Carbon Dioxide',
                    style={'width': '350px', 'margin-left': '16px'}

                ),
                html.H4('Square Size', style= {'textAlign':'left', 'margin-left': '16px'}),
                dcc.Slider(
                   id='radius1-slider',
                   min=0,
                   max=6,
                   value=1,
                   marks=size_slider_marks,
                   step=0.5

                ),
                html.H4('Opacity', style= {'textAlign':'left', 'margin-left': '16px'}),
                dcc.Slider(
                   id='opacity1-slider',
                   min=0.0,
                   max=1.0,
                   value=0.5,
                   marks=Opacity_slider_marks,
                   step=0.05

                ),
                html.Button(
                   'Download CSV',
                   id = 'download1-btn',
                   className= 'btn btn-outline-secondary',
                   style= {'margin-left': '16px'}


                ),
                dcc.Download(
                    id = 'dataframe1-down'
                ),
                html.H3('Full Emissions Map', style= {'textAlign':'center'}),
                html.Iframe(id='map1', src= None, style ={'width': '100%', 'height': '600px'}),
                html.Br(),
                html.Br(),
                html.Br()
            ])
        ]),
        dcc.Tab(label='Proximity Emissions Map', value='tab-2', children=[
            html.Div([
                html.Br(),

                html.H4('Select Vision', style= {'textAlign':'left', 'margin-left': '16px'}),
                dcc.RadioItems(

                    id      = 'vision2-radio',
                    options =  vision_options,
                    value   = 'Normal Vision',
                    className= 'btn',
                    labelClassName = 'btn btn-outline-secondary',
                    labelStyle={'display': 'inline-block', 'margin-right': '10px'}

                ),
                html.H4('Select Background Emissions Type', style= {'textAlign':'left', 'margin-left': '16px'}),
                dcc.RadioItems(

                    id      = 'bckgrndEtype2-radio',
                    options =  bckgrnd_emissions_options,
                    value   = 'Normal',
                    className= 'btn',
                    labelClassName = 'btn btn-outline-secondary',
                    labelStyle={'display': 'inline-block', 'margin-right': '10px'}

                ),
                html.H4('Select Pollutant', style= {'textAlign':'left', 'margin-left': '16px'}),
                dcc.Dropdown(
                    id='pollutant2-dropdown',
                    options=dropdown_options,
                    value='Carbon Dioxide',
                    style= {'width':'350px', 'margin-left': '16px'}
                ),
                html.H4('Square Size', style= {'textAlign':'left', 'margin-left': '16px'}),
                dcc.Slider(
                   id='radius2-slider',
                   min=0,
                   max=6,
                   value=1,
                   marks=size_slider_marks,
                   step=0.5

                ),
                html.H4('Opacity', style= {'textAlign':'left', 'margin-left': '16px'}),
                dcc.Slider(
                   id='opacity2-slider',
                   min=0.0,
                   max=1.0,
                   value=0.5,
                   marks=Opacity_slider_marks,
                   step=0.05

                ),
                html.Button(
                   'Download CSV',
                   id = 'download2-btn',
                   className= 'btn btn-outline-secondary',
                   style= {'margin-left': '16px'}


                ),
                dcc.Download(
                    id = 'dataframe2-down'
                ),
                html.H3('Proximity Emissions Map', style= {'textAlign':'center'}),
                html.Iframe(id='map2', srcDoc=None, style ={'width': '100%', 'height': '600px'}),
                html.Br(),
                html.Br(),
                html.Br()
            ])
        ]),
        dcc.Tab(label='Intersecting Squares Emission Map', value='tab-3', children=[
            html.Div([
                html.Br(),

                html.H4('Select Vision', style= {'textAlign':'left', 'margin-left': '16px'}),
                dcc.RadioItems(

                    id      = 'vision3-radio',
                    options =  vision_options,
                    value   = 'Normal Vision',
                    className= 'btn',
                    labelClassName = 'btn btn-outline-secondary',
                    labelStyle={'display': 'inline-block', 'margin-right': '10px'}

                ),
                html.H4('Select Background Emissions Type', style= {'textAlign':'left', 'margin-left': '16px'}),
                dcc.RadioItems(

                    id      = 'bckgrndEtype3-radio',
                    options =  bckgrnd_emissions_options,
                    value   = 'Normal',
                    className= 'btn',
                    labelClassName = 'btn btn-outline-secondary',
                    labelStyle={'display': 'inline-block', 'margin-right': '10px'}

                ),
                html.H4('Select Pollutant', style= {'textAlign':'left', 'margin-left': '16px'}),
                dcc.Dropdown(
                    id='pollutant3-dropdown',
                    options=dropdown_options,
                    value='Carbon Dioxide',
                    style= {'width':'350px', 'margin-left': '16px'}
                ),
                html.H4('Square Size', style= {'textAlign':'left', 'margin-left': '16px'}),
                dcc.Slider(
                   id='radius3-slider',
                   min=0,
                   max=6,
                   value=1,
                   marks=size_slider_marks,
                   step=0.5

                ),
                html.H4('Opacity', style= {'textAlign':'left', 'margin-left': '16px'}),
                dcc.Slider(
                   id='opacity3-slider',
                   min=0.0,
                   max=1.0,
                   value=0.5,
                   marks=Opacity_slider_marks,
                   step=0.05

                ),
                html.Button(
                   'Download CSV',
                   id = 'download3-btn',
                   className= 'btn btn-outline-secondary',
                   style= { 'margin-left': '16px'}

                ),
                dcc.Download(
                    id = 'dataframe3-down'
                ),
                html.H3('Intersecting Squares Emission Map', style= {'textAlign':'center'}),
                html.Iframe(id='map3', srcDoc=None, style ={'width': '100%', 'height': '600px'}),
                html.Br(),
                html.Br(),
                html.Br()
            ])
        ]),
        dcc.Tab(label='Multiline Graphs', value='tab-4', children=[
            html.Br(),

                html.H4('Select Vision', style= {'textAlign':'left', 'margin-left': '16px'}),
                dcc.RadioItems(

                    id      = 'vision4-radio',
                    options =  vision_options,
                    value   = 'Normal Vision',
                    className= 'btn',
                    labelClassName = 'btn btn-outline-secondary',
                    labelStyle={'display': 'inline-block', 'margin-right': '10px'}

                ),
                html.H4('Select Background Emissions Type', style= {'textAlign':'left', 'margin-left': '16px'}),
                dcc.RadioItems(

                    id      = 'bckgrndEtype4-radio',
                    options =  bckgrnd_emissions_options,
                    value   = 'Normal',
                    className= 'btn',
                    labelClassName = 'btn btn-outline-secondary',
                    labelStyle={'display': 'inline-block', 'margin-right': '10px'}

                ),
                html.H4('Select Emissions Type', style= {'textAlign':'left', 'margin-left': '16px'}),
                dcc.RadioItems(

                    id      = 'emissiontype4-radio',
                    options =  emission_options,
                    value   = 'Observed',
                    className= 'btn',
                    labelClassName = 'btn btn-outline-secondary',
                    labelStyle={'display': 'inline-block', 'margin-right': '10px'}

                ),
                html.H4('Select Graph Type', style= {'textAlign':'left', 'margin-left': '16px'}),
                dcc.RadioItems(

                    id      = 'graphtype4-radio',
                    options =  graph_options,
                    value   = 'Normal',
                    className= 'btn',
                    labelClassName = 'btn btn-outline-secondary',
                    labelStyle={'display': 'inline-block', 'margin-right': '10px'}

                ),
                html.H4('Select Pollutant', style= {'textAlign':'left', 'margin-left': '16px'}),
                dcc.Dropdown(
                    id='pollutant4-dropdown',
                    options=dropdown_options,
                    value='Carbon Dioxide',
                    style= {'width':'350px', 'margin-left': '16px'}
                ),
                html.H4('Square Size', style= {'textAlign':'left', 'margin-left': '16px'}),
                dcc.Slider(
                   id='radius4-slider',
                   min=0,
                   max=6,
                   value=1,
                   marks=size_slider_marks,
                   step=0.5

                ),
                dcc.Graph(
                    id = 'multilinegraph4'
                ),
                html.Div(
                    id='table4'
                ),
                html.Br(),
                html.Br()
        ]),
        dcc.Tab(label='Pie Chart', value='tab-5', children=[
            html.Br(),

                html.H4('Select Vision', style= {'textAlign':'left', 'margin-left': '16px'}),
                dcc.RadioItems(

                    id      = 'vision5-radio',
                    options =  vision_options,
                    value   = 'Normal Vision',
                    className= 'btn',
                    labelClassName = 'btn btn-outline-secondary',
                    labelStyle={'display': 'inline-block', 'margin-right': '10px'}

                ),

                html.H4('Select Pollutant', style= {'textAlign':'left', 'margin-left': '16px'}),
                dcc.Dropdown(
                    id='pollutant5-dropdown',
                    options=dropdown_options,
                    value='Carbon Dioxide',
                    style= {'width':'350px', 'margin-left': '16px'}
                ),
                html.H4('Square Size', style= {'textAlign':'left', 'margin-left': '16px'}),
                dcc.Slider(
                   id='radius5-slider',
                   min=0,
                   max=6,
                   value=1,
                   marks=size_slider_marks,
                   step=0.5

                ),
                dcc.Graph(
                    id = 'piechart5'
                ),
                html.Br(),
                html.Br()
        ]),
        dcc.Tab(label='Interactive Heatmap', style= {'textAlign':'center'}, value='tab-6', children=[
            html.Div([
                html.Br(),

                html.H4('Add new site by adding (Name, lat, lon)', style= {'textAlign':'left', 'margin-left': '16px'}),
                html.Div(
                    className = 'inputs-container',
                    style={'display':'flex', 'flex-wrap':'wrap', 'align':'left' },
                    children=[
                        html.Div(
                            className= 'input-col',
                            children =[
                                html.Label('Name', style= {'font-weight':'bold', 'margin-left':'10px'}),
                                dcc.Input(id= 'input6-name',type ='text', placeholder= 'Enter Name', className='form-control', style= {'width':'200px','margin-left':'10px','margin-right':'10px'}),
                            ]),
                        html.Div(
                            className= 'input-col',
                            children =[
                                html.Label('Latitude', style= {'font-weight':'bold', 'margin-left':'10px'}),
                                dcc.Input(id= 'input6-lat',type ='number', placeholder= 'Enter Latitude', className='form-control', style= {'width':'200px', 'margin-right':'10px'}),
                            ]),
                        html.Div(
                            className= 'input-col',
                            children =[
                                html.Label('Longitude', style= {'font-weight':'bold', 'margin-left':'10px'}),
                                dcc.Input(id= 'input6-lon',type ='number', placeholder= 'Enter Longitude', className='form-control', style= {'width':'200px', 'margin-right':'10px'}),
                            ]),


                    ]),
                html.Br(),
                html.Button(
                    'Update',
                    id='update6-button',
                    n_clicks = 0,
                    className = 'btn btn-outline-secondary',
                    style= {'width':'100px','margin-left':'15px', 'margin-right':'15px'}

                     ),
                html.Button (
                    'Reset',
                    id='reset6-button',
                    n_clicks =0,
                    className = 'btn btn-outline-secondary',
                    style= {'width':'100px', 'margin-right':'15px'}
                    ),
                html.Div(id='output6'),
                html.Br(),
                html.H4('Select Background Emissions Type', style= {'textAlign':'left', 'margin-left': '16px'}),
                dcc.RadioItems(

                    id      = 'bckgrndEtype6-radio',
                    options =  bckgrnd_emissions_options,
                    value   = 'Normal',
                    className= 'btn',
                    labelClassName = 'btn btn-outline-secondary',
                    labelStyle={'display': 'inline-block', 'margin-right': '10px'}

                ),
                html.H4('Select Pollutant', style= {'textAlign':'left', 'margin-left': '16px'}),
                dcc.Dropdown(
                    id='pollutant6-dropdown',
                    options=dropdown_options,
                    value='Carbon Dioxide',
                    style= {'width':'350px', 'margin-left': '16px'}
                ),
                html.H4('Square Size', style= {'textAlign':'left', 'margin-left': '16px'}),
                dcc.Slider(
                   id='radius6-slider',
                   min=0,
                   max=6,
                   value=1,
                   marks=size_slider_marks,
                   step=0.5

                ),
                html.H4('Opacity', style= {'textAlign':'left', 'margin-left': '16px'}),
                dcc.Slider(
                   id='opacity6-slider',
                   min=0.0,
                   max=1.0,
                   value=0.5,
                   marks=Opacity_slider_marks,
                   step=0.05

                ),
                html.Button(
                   'Download CSV',
                   id = 'download6-btn',
                   className= 'btn btn-outline-secondary',
                   style= {'margin-left': '16px'}



                ),
                dcc.Download(
                    id = 'dataframe6-down'
                ),
                html.H3('Interactive Emissions Heat Map', style= {'textAlign':'center'}),
                html.Iframe(id='map6', src= None, style ={'width': '100%', 'height': '600px'}),
                html.Br(),
                html.Br(),
                html.Br()
            ])
        ]),
    ]),
])


# app call back for updating full map
@app.callback(
    Output('map1', 'srcDoc'),

    Input('pollutant1-dropdown', 'value'),
    Input('radius1-slider', 'value'),
    Input('vision1-radio', 'value'),
    Input('opacity1-slider', 'value'),
    Input('bckgrndEtype1-radio', 'value')
)
def update_map1(pollutant, radius, vision, opacity, bckgrndEtype):
    if pollutant and radius and vision and opacity and bckgrndEtype:
        df = df_CombinedData.copy()
        map1 = create_fullmap(df, radius, pollutant, vision, opacity, bckgrndEtype)
        return map1._repr_html_()
    else:
        return no_update

# app call back for downloading dataset of full map
@app.callback(
    Output('dataframe1-down', 'data'),
    Input('pollutant1-dropdown', 'value'),
    Input('download1-btn', 'n_clicks'),
    preven_initial_call=True,
)

def downfullmap(pollutant, n_clicks):
  if pollutant and n_clicks:
    df    = df_CombinedData.copy()

    df1   = download_fullmap(df, pollutant)

    return dcc.send_data_frame (df1.to_csv, filename = f"{pollutant}(fullmap).csv")


# app call back for updating proximity map
@app.callback(
    Output('map2', 'srcDoc'),

    Input('pollutant2-dropdown', 'value'),
    Input('radius2-slider', 'value'),
    Input('vision2-radio', 'value'),
    Input('opacity2-slider', 'value'),
    Input('bckgrndEtype2-radio', 'value')
)
def update_map2(pollutant, radius, vision, opacity, bckgrndEtype):
   if pollutant and radius and vision and opacity and bckgrndEtype:
        df = df_CombinedData.copy()
        map2 = create_proximitymap(df, radius, pollutant, vision, opacity, bckgrndEtype)
        return map2._repr_html_()
   else:
        return no_update

# app call back for downloading dataset of proximity map
@app.callback(
    Output('dataframe2-down', 'data'),
    Input('pollutant2-dropdown', 'value'),
    Input('radius2-slider', 'value'),
    Input('download2-btn', 'n_clicks'),
    preven_initial_call=True,
)

def downproxmap(pollutant, radius, n_clicks):
  if pollutant and radius and n_clicks:
    df    = df_CombinedData.copy()

    df1   = download_proximitymap(df, pollutant, radius)

    return dcc.send_data_frame (df1.to_csv, filename = f"{pollutant}(proximity).csv")


# app call back for updating interesecting map

@app.callback(
    Output('map3', 'srcDoc'),

    Input('pollutant3-dropdown', 'value'),
    Input('radius3-slider', 'value'),
    Input('vision3-radio', 'value'),
    Input('opacity3-slider', 'value'),
    Input('bckgrndEtype3-radio', 'value')
)
def update_map3(pollutant, radius, vision,opacity, bckgrndEtype):
    if pollutant and radius and vision and opacity and bckgrndEtype:
        df = df_CombinedData.copy()
        map3 = create_interectingmap(df, radius, pollutant, vision,  opacity, bckgrndEtype)
        return map3._repr_html_()
    else:
        return no_update

# app call back for downloading dataset of intersecting map
@app.callback(
    Output('dataframe3-down', 'data'),
    Input('pollutant3-dropdown', 'value'),
    Input('radius3-slider', 'value'),
    Input('download3-btn', 'n_clicks'),
    preven_initial_call=True,
)

def downinsectmap(pollutant, radius, n_clicks):
  if pollutant and radius and n_clicks:
    df    = df_CombinedData.copy()

    df1   = download_intersectmap(df, pollutant, radius)

    return dcc.send_data_frame (df1.to_csv, filename = f"{pollutant}(intersect).csv")

# app call back for updating multiline graph

@app.callback(
    Output('multilinegraph4', 'figure'),

    Input('pollutant4-dropdown', 'value'),
    Input('radius4-slider', 'value'),
    Input('vision4-radio', 'value'),
    Input('bckgrndEtype4-radio', 'value'),
    Input('emissiontype4-radio', 'value'),
    Input('graphtype4-radio', 'value')
)
def update_multilinegraph4(pollutant, radius, vision, bckgrndEtype, emissiontype, graphtype):
  if pollutant and radius and vision and bckgrndEtype and emissiontype and graphtype:
    df= df_CombinedData.copy()
    df_data=data4graph_table(df,pollutant, radius, bckgrndEtype, emissiontype, graphtype)
    # Create the map using the filtered DataFrame and the specified size for the larger squares
    figure = multiline_graph(df_data, vision, graphtype)
    return figure
  else:
        return no_update

# app call back for updating table with data of multiline graph
@app.callback(
    Output('table4', 'children'),

    Input('pollutant4-dropdown', 'value'),
    Input('radius4-slider', 'value'),
    Input('bckgrndEtype4-radio', 'value'),
    Input('emissiontype4-radio', 'value'),
    Input('graphtype4-radio', 'value')
)
def update_table4(pollutant, radius, bckgrndEtype, emissiontype, graphtype):
  if pollutant and radius and bckgrndEtype and emissiontype and graphtype:
    df= df_CombinedData.copy()
    df_data=data4graph_table(df,pollutant, radius, bckgrndEtype, emissiontype, graphtype)
    # Create the map using the filtered DataFrame and the specified size for the larger squares
    table = create_table(df_data)
    return table
  else:
        return no_update

# app call back for updating piechart
@app.callback(
    Output('piechart5', 'figure'),

    Input('pollutant5-dropdown', 'value'),
    Input('radius5-slider', 'value'),
    Input('vision5-radio', 'value')
)
def update_piechart5(pollutant, radius, vision):
   if pollutant and radius and vision:
        df = df_CombinedData.copy()
        figure = pie_chart(df, pollutant,radius, vision)
        return figure
   else:
        return no_update

# app call back for updating heatmap
@app.callback(
    Output('map6', 'srcDoc'),
    Output('output6', 'children'),

    Input('pollutant6-dropdown', 'value'),
    Input('radius6-slider', 'value'),
    Input('opacity6-slider', 'value'),
    Input('bckgrndEtype6-radio', 'value'),
    Input('update6-button', 'n_clicks'),
    Input('reset6-button', 'n_clicks'),
    State('input6-name', 'value'),
    State('input6-lat', 'value'),
    State('input6-lon', 'value')

)

def update_map6(pollutant, radius,opacity, bckgrndEtype, update_clicks, reset_clicks, value1, value2, value3):
    global center_points
    mapupdate= False
    df = df_CombinedData.copy()
    ctx = dash.callback_context
    button_id = ctx.triggered[0]['prop_id'].split('.')[0]

    if button_id == 'update6-button':
      if pollutant and radius and opacity and bckgrndEtype and value1 and value2 and value3:
        center_points.append({'name': value1, 'lat': value2, 'lon': value3})
        map6 = create_interactiveheatmap(df, radius, pollutant,  opacity, bckgrndEtype)
        return map6._repr_html_(), html.Div('map updated with new location')
      else:
        map6 = create_interactiveheatmap(df, radius, pollutant,  opacity, bckgrndEtype)
        return map6._repr_html_(), html.Div('please enter valid values')
    elif button_id == 'reset6-button':
        center_points = [
            {'name': 'Billingham', 'lat': 54.5845, 'lon': -1.2818},
            {'name': 'Wilton International', 'lat': 54.5885, 'lon': -1.1287},
            {'name': 'Seal Sands North Tees', 'lat': 54.6083, 'lon': -1.1828},
            {'name': 'Low Emissions Area', 'lat': 54.608, 'lon': -1.343}
        ]
        map6 = create_interactiveheatmap(df, radius, pollutant,  opacity, bckgrndEtype)
        return map6._repr_html_(), html.Div('map reset to original')
    elif pollutant and radius and opacity and bckgrndEtype:
        df = df_CombinedData.copy()
        map6 = create_interactiveheatmap(df, radius, pollutant,  opacity, bckgrndEtype)
        return map6._repr_html_(), html.Div()
    else:
        return no_update

# app call back for downloading dataset of heatmap
@app.callback(
    Output('dataframe6-down', 'data'),
    Input('pollutant6-dropdown', 'value'),
    Input('radius6-slider', 'value'),
    Input('download6-btn', 'n_clicks'),
    preven_initial_call=True,
)

def downinsectmap6(pollutant, radius, n_clicks):
  if pollutant and radius and n_clicks:
    df    = df_CombinedData.copy()

    df1   = download_intersectmap(df, pollutant, radius)

    return dcc.send_data_frame (df1.to_csv, filename = f"{pollutant}(intersect).csv")

# running the app
if __name__ == '__main__':
   app.run_server(debug=False)
